In [ ]:
# Step 1: Import Necessary Libraries
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from google.colab import drive
# Step 2: Mount Google Drive
drive.mount('/content/drive')
# Step 3: Load Dataset
data_path = "/content/drive/MyDrive/a-z_handwritten_data.csv"
data = pd.read_csv(data_path).astype('float32')
# Step 4: Split Data for Training and Testing
X = data.drop('0', axis=1)
y = data['0']
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2)
train_x = np.reshape(train_x.values, (train_x.shape[0], 28, 28))
test_x = np.reshape(test_x.values, (test_x.shape[0], 28, 28))
# Step 5: Convert Labels to Categorical Data
train_yOHE = to_categorical(train_y, num_classes=26)
test_yOHE = to_categorical(test_y, num_classes=26)
# Step 6: Reshape Train/Test Input for Model
train_X = train_x.reshape(train_x.shape[0], 28, 28, 1)
test_X = test_x.reshape(test_x.shape[0], 28, 28, 1)
# Step 7: Define and Train CNN Model
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='valid'))
model.add(MaxPool2D(pool_size=(2, 2), strides=2))
model.add(Flatten())
model.add(Dense(64, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(26, activation="softmax"))
# Compile Model
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=1, min_lr=0.0001)
early_stop = EarlyStopping(monitor='val_loss', patience=2, verbose=1)
# Train Model
history = model.fit(train_X, train_yOHE, epochs=10, validation_data=(test_X, test_yOHE),
                    callbacks=[reduce_lr, early_stop])
# Step 8: Save Model to Google Drive
model_path = '/content/drive/MyDrive/cnn_model.h5'
model.save(model_path)
# Step 9: Load the Saved Model
loaded_model = load_model(model_path)
# Step 10: Prepare Test Image for Prediction
# Define Letter Dictionary
word_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L',
             12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W',
             23: 'X', 24: 'Y', 25: 'Z'}
# Load and Preprocess the Test Image
image_path = '/content/drive/MyDrive/image_C.jpg'
img = cv2.imread(image_path)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, img_thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)
img_final = cv2.resize(img_thresh, (28, 28))
img_final = np.reshape(img_final, (1, 28, 28, 1))
# Step 11: Make Prediction
prediction = loaded_model.predict(img_final)
predicted_letter = word_dict[np.argmax(prediction)]
print("Predicted Letter:", predicted_letter)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
9312/9312 ━━━━━━━━━━━━━━━━━━━━ 522s 55ms/step - accuracy: 0.9103 - loss: 0.3529 - val_accuracy: 0.9749 - val_loss: 0.0906 - learning_rate: 0.0010
Epoch 2/10
9312/9312 ━━━━━━━━━━━━━━━━━━━━ 497s 49ms/step - accuracy: 0.9794 - loss: 0.0733 - val_accuracy: 0.9819 - val_loss: 0.0680 - learning_rate: 0.0010
Epoch 3/10
9312/9312 ━━━━━━━━━━━━━━━━━━━━ 510s 49ms/step - accuracy: 0.9836 - loss: 0.0586 - val_accuracy: 0.9823 - val_loss: 0.0688 - learning_rate: 0.0010
Epoch 4/10
9312/9312 ━━━━━━━━━━━━━━━━━━━━ 485s 48ms/step - accuracy: 0.9920 - loss: 0.0287 - val_accuracy: 0.9918 - val_loss: 0.0324 - learning_rate: 2.0000e-04
Epoch 5/10
9312/9312 ━━━━━━━━━━━━━━━━━━━━ 501s 47ms/step - accuracy: 0.9959 - loss: 0.0144 - val_accuracy: 0.9939 - val_loss: 0.0267 - learning_rate: 2.0000e-04
Epoch 6/10
9312/9312 ━━━━━━━━━━━━━━━━━━━━ 497s 47ms/step - accuracy: 0.9974 - loss: 0.0095 - val_accuracy: 0.9932 - val_loss: 0.0306 - learning_rate: 2.0000e-04
Epoch 7/10
9312/9312 ━━━━━━━━━━━━━━━━━━━━ 443s

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 165ms/step
Predicted Letter: C


In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.utils import to_categorical
from google.colab import drive
drive.mount('/content/drive')
model_path = '/content/drive/MyDrive/cnn_model.h5'
loaded_model = load_model(model_path)

# Step 10: Prepare Test Image for Prediction
# Define Letter Dictionary
word_dict = {0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L',
             12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W',
             23: 'X', 24: 'Y', 25: 'Z'}

# Load and Preprocess the Test Image
image_path = '/content/drive/MyDrive/image_C.jpg'
#image_path = '/content/drive/MyDrive/Z.jpg'
img = cv2.imread(image_path)
#img = cv2.imshow(image_path)
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, img_thresh = cv2.threshold(img_gray, 100, 255, cv2.THRESH_BINARY_INV)
img_final = cv2.resize(img_thresh, (28, 28))
img_final = np.reshape(img_final, (1, 28, 28, 1))

# Step 11: Make Prediction
prediction = loaded_model.predict(img_final)
predicted_letter = word_dict[np.argmax(prediction)]

print("Predicted Letter:", predicted_letter)

Mounted at /content/drive


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step
Predicted Letter: C
